In [1]:
import numpy as np
import pandas as pd
from sklearn.externals.joblib import dump
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Bidirectional, CuDNNGRU, SpatialDropout1D, PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from common import SEED, TARGETS

/usr/local/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib64/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
VOCABULARY_SIZE = 100000
SEQ_MAX_LEN = 200
EMBEDDINGS_FILE = "glove.840B.300d.txt"
EMBEDDINGS_SIZE = 300
WEIGHTS_CACHE = "cache/gru_glove_weights.hdf5"
VALIDATION_PRED_FILE = "cache/gru_glove_validation_pred_fold_%s.pkl"
SUBMISSION_FILE = "submissions/submission_gru_glove.csv.gz"
K_FOLDS = 10

In [3]:
np.random.seed(SEED)

In [4]:
from tensorflow.python.client import device_lib

def get_available_devices():  
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:GPU:0']


In [5]:
train = pd.read_csv("data/train_clean.csv", encoding="utf-8")
test = pd.read_csv("data/test_clean.csv", encoding="utf-8")
submission = pd.read_csv("data/sample_submission.csv", encoding="utf-8")

In [6]:
%%time
word_tokenizer = text.Tokenizer(num_words=VOCABULARY_SIZE)
word_tokenizer.fit_on_texts(train.comment_text.values)

CPU times: user 12.2 s, sys: 76 ms, total: 12.2 s
Wall time: 12.2 s


In [7]:
%%time
list_tokenized_word_train = word_tokenizer.texts_to_sequences(train.comment_text.values)
list_tokenized_word_test = word_tokenizer.texts_to_sequences(test.comment_text.values)

CPU times: user 16.7 s, sys: 68 ms, total: 16.8 s
Wall time: 16.8 s


In [8]:
%%time
x_train_word_sequences = sequence.pad_sequences(list_tokenized_word_train, maxlen=SEQ_MAX_LEN)
x_test_word_sequences = sequence.pad_sequences(list_tokenized_word_test, maxlen=SEQ_MAX_LEN)

CPU times: user 2.86 s, sys: 316 ms, total: 3.17 s
Wall time: 3.17 s


In [9]:
def load_embeddings(file):
    embeddings = {}
    f = open(file, 'r', encoding="utf-8", errors="ignore")
    for index, line in enumerate(f):
        try:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings[word] = coefs
        except Exception:
            print("Unable to parse line %d, skipping" % index)
    f.close()
    return embeddings

In [10]:
%%time
embeddings = load_embeddings(EMBEDDINGS_FILE)
print("Loaded %d word vectors" % len(embeddings))

Unable to parse line 52343, skipping
Unable to parse line 128261, skipping
Unable to parse line 151102, skipping
Unable to parse line 200668, skipping
Unable to parse line 209833, skipping
Unable to parse line 220779, skipping
Unable to parse line 253461, skipping
Unable to parse line 365745, skipping
Unable to parse line 532048, skipping
Unable to parse line 717302, skipping
Unable to parse line 994818, skipping
Unable to parse line 1123331, skipping
Unable to parse line 1148409, skipping
Unable to parse line 1352110, skipping
Unable to parse line 1499727, skipping
Unable to parse line 1533809, skipping
Unable to parse line 1899841, skipping
Unable to parse line 1921152, skipping
Unable to parse line 2058966, skipping
Unable to parse line 2165246, skipping
Loaded 2195884 word vectors
CPU times: user 2min 56s, sys: 4.58 s, total: 3min 1s
Wall time: 2min 59s


In [11]:
def compute_embedding_matrix(embeddings, word_index):
    embedding_matrix = np.zeros((VOCABULARY_SIZE, EMBEDDINGS_SIZE))
    for word, i in word_index.items():
        if i >= VOCABULARY_SIZE:
            break
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [12]:
embedding_matrix = compute_embedding_matrix(embeddings, word_tokenizer.word_index)
print(embedding_matrix.shape)

(100000, 300)


In [13]:
def build_network(embedding_matrix):
    nn = Sequential()
    nn.add(Embedding(VOCABULARY_SIZE, EMBEDDINGS_SIZE, weights=[embedding_matrix], trainable=False, input_length=SEQ_MAX_LEN))
    nn.add(SpatialDropout1D(0.2))
    nn.add(Bidirectional(CuDNNGRU(128)))
    nn.add(Dense(256))
    nn.add(PReLU())
    nn.add(Dropout(0.5))
    nn.add(Dense(6, activation="sigmoid"))
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn

In [14]:
kfold = KFold(n_splits=K_FOLDS, random_state=SEED)

In [15]:
scores = []
test_preds = []

In [16]:
for index, (train_index, test_index) in enumerate(kfold.split(x_train_word_sequences, train[TARGETS].values)):
    
    x_train, x_val = x_train_word_sequences[train_index], x_train_word_sequences[test_index]
    y_train, y_val = train[TARGETS].values[train_index], train[TARGETS].values[test_index]
    
    callbacks = [
        EarlyStopping(monitor="val_loss"),
        ModelCheckpoint(filepath=WEIGHTS_CACHE, save_best_only=True)
    ]
    
    network = build_network(embedding_matrix)
    history = network.fit(
        x_train, y_train,
        epochs=100, batch_size=32, validation_data=(x_val, y_val), verbose=1, callbacks=callbacks
    )
    
    network.load_weights(WEIGHTS_CACHE)
    
    y_pred = network.predict_proba(x_val)
    dump(y_pred, VALIDATION_PRED_FILE % index)
    
    score = roc_auc_score(y_val, y_pred)
    print("Validation ROC-AUC score: %0.4f" % score)
    scores.append(score)
    
    test_preds.append(network.predict_proba(x_test_word_sequences))

Train on 143613 samples, validate on 15958 samples
Epoch 1/100
143613/143613 [==============================] - 308s 2ms/step - loss: 0.0524 - acc: 0.9809 - val_loss: 0.0419 - val_acc: 0.9834
Epoch 2/100
143613/143613 [==============================] - 308s 2ms/step - loss: 0.0432 - acc: 0.9833 - val_loss: 0.0405 - val_acc: 0.9840
Epoch 3/100
143613/143613 [==============================] - 307s 2ms/step - loss: 0.0403 - acc: 0.9842 - val_loss: 0.0401 - val_acc: 0.9841
Epoch 4/100
143613/143613 [==============================] - 308s 2ms/step - loss: 0.0379 - acc: 0.9849 - val_loss: 0.0404 - val_acc: 0.9840
Validation ROC-AUC score: 0.9877
Train on 143614 samples, validate on 15957 samples
Epoch 1/100
143614/143614 [==============================] - 307s 2ms/step - loss: 0.0526 - acc: 0.9808 - val_loss: 0.0440 - val_acc: 0.9828
Epoch 2/100
143614/143614 [==============================] - 308s 2ms/step - loss: 0.0432 - acc: 0.9834 - val_loss: 0.0414 - val_acc: 0.9833
Epoch 3/100
143614/

In [17]:
print("Average ROC-AUC: %0.4f" % np.mean(scores)) # 0.9868

Average ROC-AUC: 0.9880


In [18]:
for index, target in enumerate(TARGETS):
    y = 0
    for fold in range(0, K_FOLDS):
        y = y + test_preds[fold][:, index]
    submission[target] = y / K_FOLDS

In [19]:
submission.to_csv(SUBMISSION_FILE, index=False, encoding="utf-8", compression="gzip") # Scores 0.9846